<a href="https://colab.research.google.com/github/e96031413/TensorFlow_Learning_2020/blob/master/07_%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

seed = 7
np.random.seed(seed)
# 載入資料集
df = pd.read_csv("./titanic_data.csv")
# 刪除不需要的欄位
df = df.drop(["name", "ticket", "cabin"], axis=1)
# 處理遺失資料
df[["age"]] = df[["age"]].fillna(value=df[["age"]].mean())
df[["fare"]] = df[["fare"]].fillna(value=df[["fare"]].mean())
df[["embarked"]] = df[["embarked"]].fillna(value=df["embarked"].
                   value_counts().idxmax())
print(df["embarked"].value_counts())
print(df["embarked"].value_counts().idxmax())
# 轉換分類資料
df["sex"] = df["sex"].map( {"female": 1, "male": 0} ).astype(int)
# Embarked欄位的One-hot編碼
enbarked_one_hot = pd.get_dummies(df["embarked"], prefix="embarked")
df = df.drop("embarked", axis=1)
df = df.join(enbarked_one_hot)
# 將標籤的 survived 欄位移至最後
df_survived = df.pop("survived") 
df["survived"] = df_survived
print(df.head())
df.head().to_html("Ch6_2_2.html")
# 分割成訓練(80%)和測試(20%)資料集
mask = np.random.rand(len(df)) < 0.8
df_train = df[mask]
df_test = df[~mask]
print("Train:", df_train.shape)
print("Test:", df_test.shape)
# 儲存處理後的資料
df_train.to_csv("titanic_train.csv", index=False)
df_test.to_csv("titanic_test.csv", index=False)

S    916
C    270
Q    123
Name: embarked, dtype: int64
S
   pclass  sex      age  sibsp  ...  embarked_C  embarked_Q  embarked_S  survived
0       1    1  29.0000      0  ...           0           0           1         1
1       1    0   0.9167      1  ...           0           0           1         1
2       1    1   2.0000      1  ...           0           0           1         0
3       1    0  30.0000      1  ...           0           0           1         0
4       1    1  25.0000      1  ...           0           0           1         0

[5 rows x 10 columns]
Train: (1045, 10)
Test: (264, 10)


In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

seed = 7
np.random.seed(seed)
# 載入Titanic的訓練和測試資料集
df_train = pd.read_csv("./titanic_train.csv")
df_test = pd.read_csv("./titanic_test.csv")
dataset_train = df_train.values
dataset_test = df_test.values
# 分割成特徵資料和標籤資料
X_train = dataset_train[:, 0:9]
Y_train = dataset_train[:, 9]
X_test = dataset_test[:, 0:9]
Y_test = dataset_test[:, 9]
# 特徵標準化
X_train -= X_train.mean(axis=0)
X_train /= X_train.std(axis=0)
X_test -= X_test.mean(axis=0)
X_test /= X_test.std(axis=0)
# 定義模型
model = Sequential()
model.add(Dense(11, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(11, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
# 編譯模型
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
# 訓練模型
print("Training ...")
model.fit(X_train, Y_train, epochs=18, batch_size=10, verbose=0)
# 評估模型
print("\nTesting ...")
loss, accuracy = model.evaluate(X_train, Y_train)
print("訓練資料集的準確度 = {:.2f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test)
print("測試資料集的準確度 = {:.2f}".format(accuracy))
# 儲存Keras模型
print("Saving Model: titanic.h5 ...")
model.save("titanic.h5")

Using TensorFlow backend.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Training ...









Testing ...
1045/1045 [==============================] - 0s 52us/step
訓練資料集的準確度 = 0.81
264/264 [==============================] - 0s 43us/step
測試資料集的準確度 = 0.82
Saving Model: titanic.h5 ...


In [4]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.models import load_model

seed = 7
np.random.seed(seed)
# 載入Titanic的測試資料集
df_test = pd.read_csv("./titanic_test.csv")
dataset_test = df_test.values
# 分割成特徵資料和標籤資料
X_test = dataset_test[:, 0:9]
Y_test = dataset_test[:, 9]
# 特徵標準化
X_test -= X_test.mean(axis=0)
X_test /= X_test.std(axis=0)
# 建立Keras的Sequential模型
model = Sequential()
model = load_model("titanic.h5")
# 編譯模型
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
# 評估模型
print("\nTesting ...")
loss, accuracy = model.evaluate(X_test, Y_test)
print("測試資料集的準確度 = {:.2f}".format(accuracy))
# 計算分類的預測值
print("\nPredicting ...")
Y_pred = model.predict_classes(X_test)
print(Y_pred[:,0])
print(Y_test.astype(int))
# 顯示混淆矩陣
tb = pd.crosstab(Y_test.astype(int), Y_pred[:,0],
                 rownames=["label"], colnames=["predict"])
print(tb)
tb.to_html("Ch6_2_4.html")


Testing ...
264/264 [==============================] - 0s 625us/step
測試資料集的準確度 = 0.82

Predicting ...
[1 1 0 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 1 1 1 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0
 0 0 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 0 0 1 0 0 0
 1 1 0 1 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 1 1 1 0 0 0
 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1
 0 0 1 1 0]
[0 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0
 1 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0
 1 1 0 1 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 1 0 0 1
 0 1 1 0 0 1 1 0 0 1 1 1 0 0 0 1 1 1 0 1 0 0 1 0 0 1 0 1 0 1 1 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0